In [3]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from skimage import exposure

In [12]:
img1 = cv.imread("./403477.jpg")
a = 1.0
b = 0.0


gamma_arr_1 = np.linspace(0.1, 3, 100)
gamma_arr_2 = np.linspace(3, 0.1, 100)
gamma_arr = np.concatenate((gamma_arr_1, gamma_arr_2))
def write(filename, img1, fps):
    fourcc = cv.VideoWriter_fourcc('M','J','P','G')
    out = cv.VideoWriter(filename, fourcc, fps, (1920, 1080))

    for gamma in gamma_arr:
        img_gamma = ((a*((img1 / 255) ** gamma) + b) * 255).astype(np.uint8)
        out.write(img_gamma)

    out.release()

write("./test3.avi", img1, 20)

In [4]:
def cdf(im):
    c, b = cumulative_distribution(im)
    for i in range(b[0]):
        c = np.insert(c, 0, 0)
    for i in range(b[-1] + 1, 256):
        c = np.append(c, 1)
    return c

def hist_matching(c, c_t, im):
    b = np.interp(c, c_t, np.arange(256))
    pix_repl =  {i: b[i] for i in range(256)}
    mp = np.arange(0, 256)
    for (k, v) in pix_repl.items():
        mp[k] = v
    s = im.shape
    im = np.reshape(mp[im.ravel()], im.shape)
    im = np.reshape(im, s)
    return im


img1 = cv.imread("./403477.jpg")
height, width, channels = img1.shape
img2 = cv.imread("./463601.jpg")

img1 = cv.cvtColor(img1, cv.COLOR_BGR2RGB)
img2 = cv.cvtColor(img2, cv.COLOR_BGR2RGB)


cdf_im1_r = cdf(img1[:, :, 0])
cdf_im1_g = cdf(img1[:, :, 1])
cdf_im1_b = cdf(img1[:, :, 2])

cdf_im2_r = cdf(img2[:, :, 0])
cdf_im2_g = cdf(img2[:, :, 1])
cdf_im2_b = cdf(img2[:, :, 2])

im_result = np.zeros(shape=(height, width, channels), dtype=np.uint8)

im_result[:, :, 0] = hist_matching(cdf_im1_r, cdf_im2_r, img1[:, :, 0])
im_result[:, :, 1] = hist_matching(cdf_im1_g, cdf_im2_g, img1[:, :, 1])
im_result[:, :, 2] = hist_matching(cdf_im1_b, cdf_im2_b, img1[:, :, 2])


fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(15, 15))

ax[0, 0].imshow(img1)
ax[0, 0].set_title("Input Image")
plot_img_histogram(img1, ax[0, 1])
ax[0, 1].set_title("Hist Original Image")


ax[1, 0].imshow(img2)
ax[1, 0].set_title("Template Image")
plot_img_histogram(img2, ax[1, 1])

ax[2, 0].imshow(im_result)
ax[2, 0].set_title("Hist Matching Image")
plot_img_histogram(im_result, ax[2, 1])
ax[2, 1].set_title("Hist Matching Image")

NameError: name 'cumulative_distribution' is not defined